In [1]:
import tensorflow as tf
import os
import numpy as np
from matplotlib import pyplot as plt
from tensorflow.keras import layers, models

2024-10-31 00:46:22.381459: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-10-31 00:46:23.303595: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


For when doing GPU work

In [45]:
print(tf.config.list_physical_devices('GPU'))

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


# **UNET**

### **Setting up Data**

In [50]:
from sklearn.model_selection import train_test_split

def load_image_mask(image_path, mask_path):
    image = tf.io.read_file(image_path)
    image = tf.image.decode_jpeg(image, channels=3)
    image = tf.image.convert_image_dtype(image, tf.float64)  # Normalize to [0, 1]
    image = tf.image.resize(image, [128, 128])  # Resize to a smaller dimension
    mask = tf.io.read_file(mask_path)
    mask = tf.image.decode_jpeg(mask, channels=1)  # Assuming masks are single-channel
    mask = tf.image.convert_image_dtype(mask, tf.float64)  # Normalize to [0, 1]
    mask = tf.image.resize(mask, [128, 128])    # Resize to a smaller dimension
    return image, mask

def get_image_mask_paths(dataset_path):
    images = []
    masks = []
    for label in ['Positive', 'Negative']:
        image_dir = os.path.join(dataset_path, label, 'Images')
        mask_dir = os.path.join(dataset_path, label, 'Masks')

        for image_file in os.listdir(image_dir):
            if image_file.endswith('.jpg'):
                images.append(os.path.join(image_dir, image_file))
                masks.append(os.path.join(mask_dir, image_file))

    return images, masks

dataset_path = 'archive/Concrete'   # Change to your dataset path
images, masks = get_image_mask_paths(dataset_path)

# Ensure lists are of the same length
assert len(images) == len(masks), "Images and masks must have the same length"

# Split the dataset into training and validation sets (80-20 split)
train_images, val_images, train_masks, val_masks = train_test_split(images, masks, test_size=0.2, random_state=42)

def image_mask_generator(image_paths, mask_paths):
    for img_path, mask_path in zip(image_paths, mask_paths):
        yield load_image_mask(img_path, mask_path)

train_dataset = tf.data.Dataset.from_generator(
    lambda: image_mask_generator(train_images, train_masks),
    output_signature=(
        tf.TensorSpec(shape=(128, 128, 3), dtype=tf.float64),
        tf.TensorSpec(shape=(128, 128, 1), dtype=tf.float64)
    )
)
train_dataset = train_dataset.batch(32).prefetch(tf.data.AUTOTUNE)  # Adjust batch size as needed

val_dataset = tf.data.Dataset.from_generator(
    lambda: image_mask_generator(val_images, val_masks),
    output_signature=(
        tf.TensorSpec(shape=(128, 128, 3), dtype=tf.float64),
        tf.TensorSpec(shape=(128, 128, 1), dtype=tf.float64)
    )
)
val_dataset = val_dataset.batch(32).prefetch(tf.data.AUTOTUNE)  # Adjust batch size as needed

### **Model Buliding**

In [51]:
def unet(input_shape):
    inputs = tf.keras.Input(shape=input_shape)

    # Encoder
    c1 = tf.keras.layers.Conv2D(64, (3, 3), activation='relu', padding='same')(inputs)
    c1 = tf.keras.layers.Conv2D(64, (3, 3), activation='relu', padding='same')(c1)
    p1 = tf.keras.layers.MaxPooling2D((2, 2))(c1)

    c2 = tf.keras.layers.Conv2D(128, (3, 3), activation='relu', padding='same')(p1)
    c2 = tf.keras.layers.Conv2D(128, (3, 3), activation='relu', padding='same')(c2)
    p2 = tf.keras.layers.MaxPooling2D((2, 2))(c2)

    c3 = tf.keras.layers.Conv2D(256, (3, 3), activation='relu', padding='same')(p2)
    c3 = tf.keras.layers.Conv2D(256, (3, 3), activation='relu', padding='same')(c3)
    p3 = tf.keras.layers.MaxPooling2D((2, 2))(c3)

    c4 = tf.keras.layers.Conv2D(512, (3, 3), activation='relu', padding='same')(p3)
    c4 = tf.keras.layers.Conv2D(512, (3, 3), activation='relu', padding='same')(c4)
    p4 = tf.keras.layers.MaxPooling2D((2, 2))(c4)

    # Bottleneck
    c5 = tf.keras.layers.Conv2D(1024, (3, 3), activation='relu', padding='same')(p4)
    c5 = tf.keras.layers.Conv2D(1024, (3, 3), activation='relu', padding='same')(c5)

    # Decoder
    u6 = tf.keras.layers.Conv2DTranspose(512, (2, 2), strides=(2, 2), padding='same')(c5)
    u6 = tf.keras.layers.concatenate([u6, c4])
    c6 = tf.keras.layers.Conv2D(512, (3, 3), activation='relu', padding='same')(u6)
    c6 = tf.keras.layers.Conv2D(512, (3, 3), activation='relu', padding='same')(c6)

    u7 = tf.keras.layers.Conv2DTranspose(256, (2, 2), strides=(2, 2), padding='same')(c6)
    u7 = tf.keras.layers.concatenate([u7, c3])
    c7 = tf.keras.layers.Conv2D(256, (3, 3), activation='relu', padding='same')(u7)
    c7 = tf.keras.layers.Conv2D(256, (3, 3), activation='relu', padding='same')(c7)

    u8 = tf.keras.layers.Conv2DTranspose(128, (2, 2), strides=(2, 2), padding='same')(c7)
    u8 = tf.keras.layers.concatenate([u8, c2])
    c8 = tf.keras.layers.Conv2D(128, (3, 3), activation='relu', padding='same')(u8)
    c8 = tf.keras.layers.Conv2D(128, (3, 3), activation='relu', padding='same')(c8)

    u9 = tf.keras.layers.Conv2DTranspose(64, (2, 2), strides=(2, 2), padding='same')(c8)
    u9 = tf.keras.layers.concatenate([u9, c1])
    c9 = tf.keras.layers.Conv2D(64, (3, 3), activation='relu', padding='same')(u9)
    c9 = tf.keras.layers.Conv2D(64, (3, 3), activation='relu', padding='same')(c9)

    outputs = tf.keras.layers.Conv2D(1, (1, 1), activation='sigmoid')(c9)  # For binary segmentation

    model = tf.keras.Model(inputs=[inputs], outputs=[outputs])
    return model


In [52]:
input_shape = (128, 128, 3)  # Adjust based on your input size
model = unet(input_shape)
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model.summary()

Model: "model_15"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_16 (InputLayer)       [(None, 128, 128, 3)]        0         []                            
                                                                                                  
 conv2d_285 (Conv2D)         (None, 128, 128, 64)         1792      ['input_16[0][0]']            
                                                                                                  
 conv2d_286 (Conv2D)         (None, 128, 128, 64)         36928     ['conv2d_285[0][0]']          
                                                                                                  
 max_pooling2d_60 (MaxPooli  (None, 64, 64, 64)           0         ['conv2d_286[0][0]']          
 ng2D)                                                                                     

In [ ]:
steps_per_epoch = len(train_images) // 32  # Change batch size accordingly
validation_steps = len(val_images) // 32  # Change batch size accordingly

history = model.fit(train_dataset, steps_per_epoch=steps_per_epoch, validation_data=val_dataset, validation_steps=validation_steps, epochs=20)

Epoch 1/20


      8/Unknown - 4s 404ms/step - loss: 0.1119 - accuracy: 0.9542

KeyboardInterrupt: 

In [ ]:
fig = plt.figure()
plt.plot(hist.history['loss'], color = 'teal', label = 'loss')
plt.plot(hist.history['val_loss'], color = 'orange', label = 'val_loss')
fig.suptitle("Loss", fontsize = 20)
plt.legend(loc="upper left")
plt.show()

In [ ]:
fig2 = plt.figure()
plt.plot(hist.history['accuracy'], color = 'teal', label = 'accuracy')
plt.plot(hist.history['val_accuracy'], color = 'orange', label = 'val_accuracy')
fig2.suptitle("Accuracy", fontsize = 20)
plt.legend(loc="upper left")
plt.show()

### **Visualising Results on a sample image**

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Function to predict and visualize results
def visualize_results(model, dataset, num_images=5):
    plt.figure(figsize=(15, num_images * 3))
    
    for i, (images, masks) in enumerate(dataset.take(num_images)):
        preds = model.predict(images)
        
        # Select the first image and mask for visualization
        original_image = images[0].numpy()
        true_mask = masks[0].numpy().squeeze()  # Remove the channel dimension
        predicted_mask = preds[0].squeeze()     # Remove the channel dimension

        # Plot original image, true mask, and predicted mask
        plt.subplot(num_images, 3, i * 3 + 1)
        plt.imshow(original_image)
        plt.title("Original Image")
        plt.axis("off")

        plt.subplot(num_images, 3, i * 3 + 2)
        plt.imshow(true_mask, cmap='gray')
        plt.title("True Mask")
        plt.axis("off")

        plt.subplot(num_images, 3, i * 3 + 3)
        plt.imshow(predicted_mask, cmap='gray')
        plt.title("Predicted Mask")
        plt.axis("off")

    plt.tight_layout()
    plt.show()

# Call the function to visualize results
visualize_results(model, val_dataset)

AttributeError: 'str' object has no attribute 'numpy'